In [1]:
import os
import glob
import cudf
import dask_cudf
import hydra

In [2]:
# Load hydra configuration
with hydra.initialize(version_base=None, config_path="../../../aiagents4pharma/talk2knowledgegraphs/configs"):
    cfg = hydra.compose(
        config_name="config", overrides=["tools/multimodal_subgraph_extraction=default"]
    )
    cfg = cfg.tools.multimodal_subgraph_extraction
cfg

{'_target_': 'talk2knowledgegraphs.tools.multimodal_subgraph_extraction', 'ollama_embeddings': ['nomic-embed-text'], 'temperature': 0.1, 'streaming': False, 'topk': 5, 'topk_e': 5, 'cost_e': 0.5, 'c_const': 0.01, 'root': -1, 'num_clusters': 1, 'pruning': 'gw', 'verbosity_level': 0, 'node_id_column': 'node_id', 'node_attr_column': 'node_attr', 'edge_src_column': 'edge_src', 'edge_attr_column': 'edge_attr', 'edge_dst_column': 'edge_dst', 'node_colors_dict': {'gene/protein': '#6a79f7', 'molecular_function': '#82cafc', 'cellular_component': '#3f9b0b', 'biological_process': '#c5c9c7', 'drug': '#c4a661', 'disease': '#80013f'}, 'biobridge': {'source': 'aiagents4pharma/talk2knowledgegraphs/tests/files/biobridge_multimodal/', 'node_type': ['gene/protein', 'molecular_function', 'cellular_component', 'biological_process', 'drug', 'disease']}}

In [3]:
# Loop over nodes and edges
graph_dict = {}
for element in ["nodes", "edges"]:
    # Make an empty dictionary for each folder
    graph_dict[element] = {}
    for stage in ["enrichment"]:
        # print(key, path, stage)
        # Create the file pattern for the current subfolder
        file_list = glob.glob(os.path.join('../../../', 
                                           cfg.biobridge.source, 
                                           element, 
                                           stage, '*.parquet.gzip'))
        # if element != "edges" and stage == "embedding":
        # Read and concatenate all dataframes in the folder
        graph_dict[element][stage] = cudf.concat([cudf.read_parquet(f) for f in file_list], ignore_index=True)

In [4]:
graph_dict["nodes"]["enrichment"].head()

,node_index,node_id,node_name,node_type,desc,feat
0,45939,methyltransferase activity_(53517),methyltransferase activity,molecular_function,methyltransferase activity belongs to molecula...,Catalysis of the transfer of a methyl group to...
1,45940,"catalytic activity, acting on a tRNA_(53518)","catalytic activity, acting on a tRNA",molecular_function,"catalytic activity, acting on a tRNA belongs t...",Catalytic activity that acts to modify a tRNA.
2,45941,"catalytic activity, acting on DNA_(53519)","catalytic activity, acting on DNA",molecular_function,"catalytic activity, acting on DNA belongs to m...",Catalytic activity that acts to modify DNA.
3,45942,"catalytic activity, acting on a protein_(53520)","catalytic activity, acting on a protein",molecular_function,"catalytic activity, acting on a protein belong...",Catalytic activity that acts to modify a protein.
4,45943,"catalytic activity, acting on RNA_(53521)","catalytic activity, acting on RNA",molecular_function,"catalytic activity, acting on RNA belongs to m...","Catalytic activity that acts to modify RNA, dr..."
